<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/vector_stores/FirestoreVectorStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Firestore Vector Store

# Google Firestore (Native Mode)

> [Firestore](https://cloud.google.com/firestore) is a serverless document-oriented database that scales to meet any demand. Extend your database application to build AI-powered experiences leveraging Firestore's Langchain integrations.

This notebook goes over how to use [Firestore](https://cloud.google.com/firestore) to to store vectors and query them using the `FirestoreVectorStore` class.

## Before You Begin

To run this notebook, you will need to do the following:

* [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
* [Enable the Firestore API](https://console.cloud.google.com/flows/enableapi?apiid=firestore.googleapis.com)
* [Create a Firestore database](https://cloud.google.com/firestore/docs/manage-databases)

After confirmed access to database in the runtime environment of this notebook, filling the following values and run the cell before running example scripts.

In [ ]:
# @markdown Please specify a source for demo purpose.
COLLECTION_NAME = (
    "test_collection"  # @param {type:"CollectionReference"|"string"}
)

## Library Installation

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙. For this notebook, we will also install `langchain-google-genai` to use Google Generative AI embeddings.

In [ ]:
%pip install --quiet llama-index
%pip install --quiet llama-index-vector-stores-firestore langchain-google-vertexai

### ☁ Set Your Google Cloud Project
Set your Google Cloud project so that you can leverage Google Cloud resources within this notebook.

If you don't know your project ID, try the following:

* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "extensions-testing"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 🔐 Authentication

Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

- If you are using Colab to run this notebook, use the cell below and continue.
- If you are using Vertex AI Workbench, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth

auth.authenticate_user()

# Basic Usage

### Initialize FirestoreVectorStore

`FirestoreVectroStore` allows you to store new vectors in a Firestore database. You can use it to store embeddings from any model, including those from Google Generative AI.

In [ ]:
from llama_index.vector_store.firestore import FirestoreVectorStore

from google.api_core.client_options import ClientOptions
from google.cloud import firestore

client_options = ClientOptions()
client = firestore.Client(client_options=client_options)

store = FirestoreVectorStore(
    client=client,
    collection_name=COLLECTION_NAME,
)

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest",
    project=PROJECT_ID,
)

### Add vectors

In [ ]:
from llama_index.core.schema import TextNode

store.add(
    [
        TextNode(
            id_="apple",  # unique identifier for the node
            embedding=embedding.embed_query("apple"),
            text="apple",
        ),
    ]
)

### Perform search

You can use the `FirestoreVectorStore` to perform similarity searches on the vectors you have stored. This is useful for finding similar documents or text.

In [ ]:
from llama_index.core.vector_stores.types import (
    VectorStoreQuery,
)

query = "I like fuji"
embeddings = embedding.embed_query(query)

store.query(VectorStoreQuery(query_embedding=embeddings))